In [1]:
import sys, os, json
from time import time, strftime, localtime
import cPickle as pickle
from ModelExRecCNOT import *
from ModelSurface1EC import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('../../Param/PureError/Surface_1EC_D5/2017-12-26-15-55-31.json') as paramfile:
    param = json.load(paramfile)

In [3]:
param

{u'data': {u'num test batch': 10, u'num trials': 10, u'test fraction': 0.1},
 u'env': {u'EC scheme': u'SurfaceD5',
  u'FT scheme': u'Surface1EC',
  u'look up': False,
  u'param folder': u'../../Param/PureError/Surface_1EC_D5/',
  u'pickle folder': u'../../Data/Pkl/PureError/Surface_1EC_D5/e-03/',
  u'raw folder': u'../../Data/Compact/Surface_1EC_D5/e-03/',
  u'report folder': u'../../Reports/PureError/Surface_1EC_D5/'},
 u'nn': {u'W std': 0.001351297996828406,
  u'b std': 0.0,
  u'iso': False,
  u'keep rate': 0.554206472588703,
  u'kernel size': 3,
  u'mixed': False,
  u'num filters': 49,
  u'num hidden': [482],
  u'padding size': 1,
  u'type': u'3DCNN'},
 u'opt': {u'batch size': 2000,
  u'decay': 0.9876636869768898,
  u'iterations': 20,
  u'learning rate': 0.00012062901951802724,
  u'momentum': 0.8074066502068599,
  u'tune value': 0.0005000867500471026},
 u'usr': {u'verbose': False}}

In [4]:
if(param['env']['EC scheme']=='SurfaceD3'):
    import _SurfaceD3Lookup as lookup 
elif(param['env']['EC scheme']=='SurfaceD5'):
    import _SurfaceD5Lookup as lookup 
elif(param['env']['EC scheme']=='ColorD3'):
    import _ColorD3Lookup as lookup 
elif(param['env']['EC scheme']=='ColorD5'):
    import _ColorD5Lookup as lookup 
else:
    raise ValueError('Unknown circuit type.')
spec= lookup.Spec()

In [5]:
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-03/2.000e-03.pkl', 'rb') as input_file:
    m = pickle.load(input_file)

m.test_size= int(param['data']['test fraction'] * m.data_size)
m.train_size= m.data_size - m.test_size
m.num_batches= m.train_size // param['opt']['batch size']
m.spec= spec

In [6]:
n= []
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-04/3.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-04/4.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-04/5.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-04/6.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-04/7.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Surface_1EC_D5/e-04/8.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
for i in range(len(n)):
    n[i].test_size= int(param['data']['test fraction'] * n[i].data_size)

In [7]:
vals= {}
for i in range(len(n)):
    vals[i]= []

In [8]:
verbose= param['usr']['verbose']
batch_size= param['opt']['batch size']
learning_rate= param['opt']['learning rate']
num_iterations= param['opt']['iterations']
momentum_val= param['opt']['momentum']
decay_rate= param['opt']['decay']

tf.reset_default_graph()
x, y, predict= {}, {}, {}
for key in m.spec.err_keys:
    with tf.variable_scope(key):
        x[key] = tf.placeholder(tf.float32, [None,m.spec.input_size])
        y[key] = tf.placeholder(tf.float32, [None,2])
keep_rate= tf.placeholder(tf.float32)
#cost= surface_conv3d_cost(param['nn'], m.spec, x, y, predict, keep_rate)
cost= m.cost_function(param['nn'], x, y, predict, keep_rate)
train = tf.train.RMSPropOptimizer(learning_rate, decay=decay_rate, \
    momentum=momentum_val).minimize(cost)
init = tf.global_variables_initializer()

In [9]:
for T in range(5):

    pointer= randint(0, m.data_size - 1) # self.test_size * trial
    t_beg= (m.train_size + pointer) % m.data_size

    predictions= []

    with tf.Session() as session:
        session.run(init)

        for i in tqdm.tqdm(range(num_iterations)):
            for j in range(m.num_batches):
                beg= (j * batch_size + pointer) % m.data_size
                feed_dict={}
                for key in m.spec.err_keys:
                    feed_dict[x[key]]= \
                        cyc_pick(m.syn[key], beg, batch_size)
                    feed_dict[y[key]]= \
                        cyc_pick(m.log_1hot[key], beg, batch_size)
                    feed_dict[keep_rate]= param['nn']['keep rate']
                session.run(train, feed_dict)

        for i in tqdm.tqdm(range(len(n))):
            prediction={}
            num_test_batches= param['data']['num test batch'] if \
                'num test batch' in param['data'].keys() else 1
            test_batch_size= m.test_size / num_test_batches
            for j in range(num_test_batches):
                beg= (j * test_batch_size + t_beg) % m.data_size
                if j==num_test_batches-1:
                    test_batch_size+= n[i].test_size % num_test_batches
                feed_dict={}
                for key in m.spec.err_keys:
                    feed_dict[x[key]]= \
                        cyc_pick(n[i].syn[key], beg, test_batch_size)
                feed_dict[keep_rate]= 1.0
                for key in m.spec.err_keys:
                    res= session.run(predict[key], feed_dict)
                    if key in prediction.keys():
                        prediction[key]= np.append(prediction[key], res, axis=0)
                    else:
                        prediction[key]= res
            predictions.append(prediction)

    for i in tqdm.tqdm(range(len(n))):
        vals[i].append(n[i].error_scale * n[i].num_logical_fault(predictions[i], t_beg))
    
    vals

100%|██████████| 6/6 [03:37<00:00, 36.33s/it]


In [10]:
vals

{0: [0.00012804097214566765,
  0.00010191016150369467,
  0.00012085499921912509,
  0.00011236248576048386,
  9.603072910925074e-05],
 1: [0.00019456424589595452,
  0.00021077793305395072,
  0.00022016480456647487,
  0.00023467178781310307,
  0.00024747206714836325],
 2: [0.00039245825140643467,
  0.0003674077247209176,
  0.0003621888649947682,
  0.00036845149666614745,
  0.0003632326369399981],
 3: [0.0005836409321459078,
  0.0005652874437136466,
  0.0005922058934142964,
  0.000572628839086551,
  0.0005934294593097804],
 4: [0.0008802429600000476,
  0.0008634230308280722,
  0.0008746363169427225,
  0.000891456246114698,
  0.0008508080839490907],
 5: [0.0012285256706001228,
  0.001233238684681709,
  0.0012678007879466742,
  0.0012772268161098462,
  0.001237951698763295]}